# Visualizing File Changes Across Commits

This notebook demonstrates the MultiCommitCompare Class with proper pandas display configuration.

In [1]:
from tardisbase.testing.regression_comparison.run_tests import run_tests
from tardisbase.testing.regression_comparison.visualize_files import MultiCommitCompare
from tardisbase.testing.regression_comparison.util import get_last_n_commits
import pandas as pd
from IPython.display import display

In [2]:
# Configure pandas display options for better visualization
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Configuration for the analysis
config = {
    "tardis_repo_path": "/home/riddhigangbhoj/tardis-work/tardis",
    "regression_data_repo_path": "/home/riddhigangbhoj/tardis-work/tardis-regression-data",
    "branch": "master",
    "n": 3, # Last n commits
    # "commits": ["300e565e83112528faaa76e970057ffb1b13f743", "2a06fdfb60190bbd9b49ff572d78772607138660", "2d775dcd1c486227532f537fc41066e942000e56"],  # Uncomment for specific commits
    # "commits": commits.calculate_commits(n=10, gap=0, info=False), #Uncomment for n commits with custom gap
    "use_conda": True,
    "conda_manager": "conda"
}

## Option 1: Run tests to generate regression data commits

### Case 1: Test latest N TARDIS commits

In [4]:
processed_commits, regression_commits, original_head = run_tests(
        config["tardis_repo_path"], 
        config["regression_data_repo_path"], 
        config["branch"],
        n=config["n"], 
        use_conda=config["use_conda"],
        conda_manager=config["conda_manager"],
        force_recreate=True,
        test_path="tardis",
        default_curr_env="/home/riddhigangbhoj/miniforge3/envs/tardis-master"
    )

Original HEAD of regression data repo: cdafbf1ae780f24591444c63d1bbc8b41404244e
Processing commit 1/3: 701bb18916886ecf2797b0dda4843750f69592da
Creating conda environment: tardis-test-701bb189
Checking if environment tardis-test-701bb189 exists...
Environment tardis-test-701bb189 exists, removing it for recreation...
Creating conda environment: conda create --name tardis-test-701bb189 --file /tmp/tmpltdyxlbi.lock -y
Installing TARDIS with all extras ['viz', 'tardisbase']: conda run -n tardis-test-701bb189 pip install -e /home/riddhigangbhoj/tardis-work/tardis[viz,tardisbase]

=== Phase 1: Running 'not continuum' tests for commit 701bb18916886ecf2797b0dda4843750f69592da ===
Running Not continuum tests: conda run -n tardis-test-701bb189 python -m pytest tardis --tardis-regression-data=/home/riddhigangbhoj/tardis-work/tardis-regression-data --generate-reference --disable-warnings -vvv -m not continuum

=== Phase 2: Running 'continuum' tests for commit 701bb18916886ecf2797b0dda4843750f6959

### Case 2: Test specific TARDIS commits
When `commits_input` is a list of commit hashes

In [5]:
# processed_commits, regression_commits, original_head = run_tests(
#     config["tardis_repo_path"], 
#     config["regression_data_repo_path"], 
#     config["branch"], 
#     commits_input=config["commits"],
#     use_conda=config["use_conda"],
#     conda_manager=config["conda_manager"]
# )

## Option 2: Use existing regression data commits

### A.  Manual Commit Selection

In [6]:
# regression_commits = ["66a96a847c873544babb7bf934040c86433a5962",
#                       "d12d869bd2bb2038c9090852ee9ef998959f412d",
#                       "b008a7180440a697ad5b54a9f77b692d4f71b120",
#                       "a2a946a43d710c44bb3b08bcae69359fe13ed032",
#                       "9404dc594563d9457e3ba91fcaa8400cae231801"]

### B.  Automatically fetch the most recent N commits from regression data repository

In [7]:
# regression_commits = get_last_n_commits(n=2, repo_path=config["regression_data_repo_path"])
# regression_commits

## Visualize File Changes
Create a visualizer object to analyze file changes across commits.

Case 1: Direct regression data commits (no TARDIS commits)

In [8]:
# visualizer = MultiCommitCompare(
#     regression_repo_path=config["regression_data_repo_path"],
#     commits=regression_commits,
#     # file_extensions=('.h5', '.hdf5') # Uncomment to filter specific files
# )


Case 2: Regression data commits generated from TARDIS commits

In [9]:
visualizer = MultiCommitCompare(
    regression_repo_path=config["regression_data_repo_path"],
    commits=regression_commits,
    tardis_commits=processed_commits,
    tardis_repo_path=config["tardis_repo_path"],
    # file_extensions=('.h5', '.hdf5') # Uncomment to filter specific files
    compare_function="git_diff"
)

In [10]:
# Analyze the commits
visualizer.analyze_commits()

Analyzing 3 commits (2 transitions)...
Processing transition 1/2: 9e9073-3b4b85
Processing transition 2/2: 1d0638-9e9073
Found 339 total files across all transitions.


### Display the file change matrix 

In [11]:
commit_info, legend, matrix = visualizer.get_analysis_results()

In [12]:
commit_info

,Commit #,Regression Hash,Description,Date
0,1,3b4b8524,Regression data for --Relativity BugFix [2] (#3176),2025-07-28 08:39
1,2,9e907329,Regression data for --add from workflow method to sdec and liv plot (#3198),2025-07-28 09:02
2,3,1d063881,Regression data for --Post-release 2025.07.20 (#3201),2025-07-28 09:26


In [13]:
legend

A          Added
D        Deleted
M       Modified
•      Unchanged
−    Not-Present
Name: Legend, dtype: object

In [14]:
matrix

,Files,9e9073-3b4b85,1d0638-9e9073
0,.gitattributes,•,•
1,.github/actions/setup_env/action.yml,•,•
2,.github/workflows/run-notebook.yml,•,•
3,.github/workflows/trigger-lfs-cache.yml,•,•
4,.gitignore,•,•
5,LICENSE,•,•
6,__init__.py,•,•
7,arepo_data/arepo_snapshot.hdf5,•,•
8,arepo_data/arepo_snapshot.json,•,•
9,atom_data/chianti_He.h5,•,•
